# **Global SetUp**

Packages

In [1]:
# Packages
!pip install faker
!pip install -U spacy
!python -m spacy download de_core_news_md


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 12.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Imports

In [2]:
# Imports
import torch
import re
from google.colab import files
from transformers import AutoTokenizer, AutoModelForTokenClassification
import spacy
from faker import Faker
import random

#Faker SetUp
faker = Faker("de_DE")
Faker.seed(42)

# 🧠 Spacy Setup
nlp = spacy.load("de_core_news_md")

## **Level 0**

**`All needed for Level 0)`**

Model & Tokenizer

In [3]:
model_name = "iiiorg/piiranha-v1-detect-personal-information"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Device

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

DebertaV2ForTokenClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(251000, 768, padding_idx=0)
      (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-11): 12 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=768, out_features=768, bias=True)
              (key_proj): Linear(in_features=768, out_features=768, bias=True)
              (value_proj): Linear(in_features=768, out_features=768, bias=True)
              (pos_dropout): Dropout(p=0.1, inplace=False)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

Spacing

In [5]:
def normalize_placeholders_spacing(text):
    text = re.sub(r'(\w)(\[[A-Z\-_]+])', r'\1 \2', text)  # z.B. Leo[NAME] → Leo [NAME]
    text = re.sub(r'(\])(\[)', r'\1 \2', text)            # ][ → ] [
    return text

Helpfunction 1: Iban Recognition

In [6]:
# 🚀 RegEx Patterns für IBAN, BIC, URL
def mask_regex_patterns(text):
    # IBAN
    text = re.sub(r'\bDE\d{20}\b', '[IBAN]', text)

    # BIC
    text = re.sub(r'\b[A-Z]{6}[A-Z0-9]{2}([A-Z0-9]{3})?\b', '[BIC]', text)

    # URL / Domain
    text = re.sub(r'\b(?:https?://)?(?:www\.)?[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}\b', '[URL]', text)

    return text

Helpfunction 2: Domain Specific Reg:

In [7]:
def postprocess_domain_specific_pii(text):
    # Vertragsnummer
    text = re.sub(r"(?i)(vertragsnummer\s*[:\-\s]*)\d{6,12}", r"\1[CONTRACT_NUMBER]", text)

    # Vertragskonto
    text = re.sub(r"(?i)(vertragskonto\s*[:\-\s]*)\d{6,12}", r"\1[ACCOUNT_NUMBER]", text)

    # Kundennummer
    text = re.sub(r"(?i)(kundennummer\s*[:\-\s]*)\d{6,12}", r"\1[CUSTOMER_NUMBER]", text)

    # Rechnungsnummer
    text = re.sub(r"(?i)(rechnungsnummer\s*[:\-\s]*)\d{6,20}", r"\1[INVOICE_NUMBER]", text)

    # Zählernummer (oft alphanumerisch!)
    text = re.sub(r"(?i)(zählernummer\s*[:\-\s]*)[a-zA-Z0-9]+", r"\1[METER_NUMBER]", text)

    # Zählerstand (numerisch, evtl. Komma oder Punkt)
    text = re.sub(r"(?i)(zählerstand\s*[:\-\s]*)\d+[.,]?\d*", r"\1[METER_READING]", text)

    # Verbrauchsstelle (Adresse, bis zum Zeilenumbruch)
    text = re.sub(r"(?i)(verbrauchsstelle\s*[:\-\s]*)[^\n]+", r"\1[STREET_ADDRESS]", text)

    return text


Helpfunction 3: Recognizer

In [8]:
# 🔧 Zusatz-Namenserkennung
def mask_names_with_spacy(text):
    doc = nlp(text)
    masked_text = text

    # Rückwärts ersetzen
    for ent in reversed(doc.ents):
        if ent.label_ == "PER":
            person_text = ent.text.strip()
            start, end = ent.start_char, ent.end_char

            # Prüfen: enthält Name Leerzeichen → aufteilen in Vorname + Nachname
            name_parts = person_text.split()
            if len(name_parts) == 2:
                forename, surname = name_parts

                # Ersetze Nachname zuerst (damit Offset von Vorname nicht verschoben wird)
                surname_start = text.find(surname, start)
                surname_end = surname_start + len(surname)
                masked_text = masked_text[:surname_start] + "[SURNAME]" + masked_text[surname_end:]

                forename_start = text.find(forename, start)
                forename_end = forename_start + len(forename)
                masked_text = masked_text[:forename_start] + "[FORENAME]" + masked_text[forename_end:]

            else:
                # Einzelname → einfach als [NAME]
                masked_text = masked_text[:start] + "[NAME]" + masked_text[end:]

    # Spacing normalisieren (nutzt deine bestehende Funktion)
    return normalize_placeholders_spacing(masked_text)

# **Upload Files to fetch into model**

In [9]:
# Upload & Short Preprocessing
uploaded_files = files.upload()
file_texts = {}
for filename, file_content in uploaded_files.items():
    text = file_content.decode("utf-8").replace('-', '')
    text = re.sub(r'(?<=\d) (?=\d)', '', text)
    file_texts[filename] = text

Saving 51.txt to 51.txt


# **Level 0 a) - PIIranha Base Model**

In [10]:
LABEL_TO_PLACEHOLDER = {
    "PERSON": "NAME",
    "EMAIL": "EMAIL",
    "PHONE": "PHONE",
    "IBAN": "IBAN",
    "ADDRESS": "ADDRESS",
    "LOC": "ADDRESS",
    "ORG": "ORG",
    "CONTRACT": "CONTRACT",
    "CARDINAL": "NUMBER",
    "DATE": "DATE"
}

In [11]:
def mask_pii(text, aggregate_redaction=True):
    encoded = tokenizer(text, return_offsets_mapping=True, return_tensors="pt", truncation=True, padding=True)
    offset_mapping = encoded.pop('offset_mapping')[0].tolist()
    inputs = {k: v.to(device) for k, v in encoded.items()}

    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)[0].tolist()

    redactions = []
    current_start = None
    current_end = None
    current_label = None

    for i, (offset, pred_id) in enumerate(zip(offset_mapping, predictions)):
        start, end = offset
        if start == end:
            continue  # Sondertoken (z.B. [CLS], [SEP])
        label = model.config.id2label[pred_id]
        label = label.replace("B-", "").replace("I-", "")  # Label normalisieren
        if label != 'O':
            if current_label is None:
                current_start = start
                current_end = end
                current_label = label
            elif label == current_label:
                current_end = end
            else:
                redactions.append((current_start, current_end, current_label))
                current_start, current_end, current_label = start, end, label
        else:
            if current_label is not None:
                redactions.append((current_start, current_end, current_label))
                current_start, current_end, current_label = None, None, None

    if current_label is not None:
        redactions.append((current_start, current_end, current_label))

    # Rückwärts ersetzen
    masked_text = text
    for start, end, label in sorted(redactions, reverse=True):
        placeholder = LABEL_TO_PLACEHOLDER.get(label, label)
        replacement = "[redacted]" if aggregate_redaction else f"[{placeholder}]"
        masked_text = masked_text[:start] + replacement + masked_text[end:]

    # Spacing normalisieren
    return normalize_placeholders_spacing(masked_text)

**Anonymize & Download Set For Testing**

Provides the testing set correctly named

In [14]:
def anonymize_and_download_named(files_dict):
    """
    Anonymisiert Texte mit Dateinamen und speichert sie als 'piiranha_<ID>.txt'.
    """
    for filename, text in files_dict.items():
        match = re.search(r'(\d+)', filename)
        file_id = match.group(1) if match else 'X'

        masked = mask_pii(text, aggregate_redaction=False)
        output_filename = f"piiranha_{file_id}.txt"

        with open(output_filename, "w", encoding="utf-8") as f:
            f.write(masked)
        files.download(output_filename)

In [16]:
anonymize_and_download_named(file_texts)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Printed Output Basic Anonymization Model**

In [48]:
for filename, file_content in uploaded_files.items():
    text = file_content.decode("utf-8")  # ← wichtig!

    print(f"\n📄 Original E-Mail ({filename}):\n{text}")
    print(f"\n🔐 Anonymized E-Mail (Base-Model) ({filename}):\n{mask_pii(text, aggregate_redaction=False)}")


📄 Original E-Mail (51.txt):
Sehr geehrte Damen und Herren,
hiermit lege Ich, Catharina Thies, Vertragsnummer 402157398, bei der Schlussrechnung 2022/ 2023 Wiederspruch ein.
Die Wohnungsabnahme war am 15.05.2022, anbei das Übergabeprotokoll der Hausverwaltung. Der Abbrechnungszeitraum vom 16.05.-31.05.22 fäll somit nicht mehr in meinen Bemessungszeitraum. Anbei auch die neue Meldebescheinigung. Ich bitte hiermit um Klärung der Abrechnung.
Mit freundlichen Grüßen
Catharina Thies


🔐 Anonymized E-Mail (Base-Model) (51.txt):
Sehr geehrte Damen und Herren,
hiermit lege Ich,[GIVENNAME] [SURNAME], Vertragsnummer [ACCOUNTNUM], bei der Schlussrechnung 2022/ 2023 Wiederspruch ein.
Die Wohnungsabnahme war am 15.05.2022, anbei das Übergabeprotokoll der Hausverwaltung. Der Abbrechnungszeitraum vom 16.05.-31.05.22 fäll somit nicht mehr in meinen Bemessungszeitraum. Anbei auch die neue Meldebescheinigung. Ich bitte hiermit um Klärung der Abrechnung.
Mit freundlichen Grüßen [GIVENNAME] [SURNAME]


📄 

# **Level 0 b) - Synthetic E-Mail Generation**

In [13]:
def generate_replacement_map():
    gender = random.choice(["male", "female"])
    vorname = faker.first_name_male() if gender == "male" else faker.first_name_female()
    nachname = faker.last_name()
    full_name = f"{vorname} {nachname}"
    telefonnummer = faker.phone_number()
    contract_number = f"{random.choice(['I-', 'V-'])}{faker.random_number(digits=7)}"
    iban = "DE" + str(faker.random_number(digits=20, fix_len=True))
    payment = f"{faker.random_int(min=20, max=250)}€"
    date_of_birth = faker.date_of_birth(minimum_age=18, maximum_age=99).strftime("%d.%m.%Y")
    username = faker.user_name()

    return {
        # Personen
        "NAME": full_name,
        "FORENAME": vorname,
        "GIVENNAME": vorname,
        "SURNAME": nachname,
        "DATEOFBIRTH": date_of_birth,

        # Kontakt
        "EMAIL": faker.email(),
        "USERNAME": username,
        "PHONE": telefonnummer,
        "TELEPHONENUM": telefonnummer,

        # Adresse
        "ADDRESS": faker.street_address(),
        "STREET": faker.street_name(),
        "BUILDINGNUM": str(faker.building_number()),
        "ZIPCODE": faker.postcode(),
        "CITY": faker.city(),
        "POSTAL": faker.postcode(),

        # Vertrag / Konto
        "ACCOUNTNUM": contract_number,
        "IBAN": iban,
        "PAYMENT": payment,

        # I-Variante (optional)
        "I-ACCOUNTNUM": contract_number,
        "I-STREET": faker.street_name(),
        "I-BUILDINGNUM": str(faker.building_number()),
        "I-ZIPCODE": faker.postcode(),
        "I-CITY": faker.city(),
        "I-SURNAME": nachname,
        "I-GIVENNAME": vorname,
        "I-FULLNAME": full_name,
        "I-TELEPHONENUM": telefonnummer,
    }


# 🔄 Platzhalter ersetzen
def replace_placeholders(anonymized_text, replacements):
    for placeholder, fake_value in replacements.items():
        anonymized_text = anonymized_text.replace(f"[{placeholder}]", fake_value)
        anonymized_text = anonymized_text.replace(f"<<{placeholder}>>", fake_value)
    return anonymized_text

# 🔧 Formatierung nach Einsetzen verbessern
def fix_spacing_around_replacements(text):
    text = re.sub(r'(\w)(\+[\d\(])', r'\1 \2', text)
    text = re.sub(r'(\])(\w)', r'\1 \2', text)
    text = re.sub(r'(\w)(\[)', r'\1 \2', text)
    return text


Output Synthetic Email Generation

In [ ]:
for filename, file_content in uploaded_files.items():

    print(f"\n📄 🔹 Original Text ({filename}):")
    print(text_content)

    # Anonymisieren
    anonymized = mask_pii(text_content, aggregate_redaction=False)


    print(f"\n🔐 Anonymised Text ({filename}):")
    print(anonymized)

    # Synthetische Daten erzeugen und einsetzen
    replacements = generate_replacement_map()
    synthetic_text = replace_placeholders(anonymized, replacements)
    synthetic_text = fix_spacing_around_replacements(synthetic_text)

    print(f"\n🧬 ✅ Synthetic Text ({filename}):")
    print(synthetic_text)



📄 🔹 Original Text (2022 (1).txt):


NameError: name 'text_content' is not defined

# **Level 1**

1.1 Performance Analysis - **Anonymization**

1.2 Performance - **Synthetic E-Mail Generation**

1.3 Model Improvement Recommendations

In [ ]:
# === 🔄 Vollständige PII-Anonymisierungs-Pipeline als Funktion ===

def run_full_anonymization_pipeline(text, aggregate_redaction=False):
    """
    Führt die vollständige PII-Anonymisierungspipeline aus:
    1. Regex-basierte Maskierung (IBAN, BIC, URL)
    2. Namenserkennung mit spaCy
    3. Piranha-Model-Erkennung
    4. Domain-spezifische Nachbearbeitung
    """
    # 1️⃣ Regex-basierte Maskierung
    pre_masked_text = mask_regex_patterns(text)

    # 2️⃣ Namenserkennung mit spaCy
    names_masked_text = mask_names_with_spacy(pre_masked_text)

    # 3️⃣ Piranha-Model drüber laufen lassen
    model_anonymized_text = mask_pii(names_masked_text, aggregate_redaction=aggregate_redaction)

    # 4️⃣ Domain-spezifische Nachbearbeitung
    final_anonymized_text = postprocess_domain_specific_pii(model_anonymized_text)

    return final_anonymized_text



In [ ]:

def output_anonymization_results(title, original_text, anonymized_text, save_to_file=False, output_folder="anonymized_results"):
    """
    Gibt Original- und Anonymisierungsergebnis aus.
    Optional wird das anonymisierte Ergebnis in eine Textdatei gespeichert.

    :param title: Titel oder Dateiname zur Anzeige
    :param original_text: Originaltext
    :param anonymized_text: Text nach Anonymisierung
    :param save_to_file: Wenn True, wird das Ergebnis gespeichert
    :param output_folder: Zielordner für Dateien
    """
    print(f"\n📄 {title} - Originaltext:\n{original_text}\n")
    print(f"🔐 {title} - Anonymisierter Text:\n{anonymized_text}\n")

    if save_to_file:
        # Sicherer Dateiname erzeugen
        safe_title = title.replace(" ", "_").replace("(", "").replace(")", "").replace(":", "").replace("/", "_")

        # Ordner anlegen falls nicht vorhanden
        os.makedirs(output_folder, exist_ok=True)

        # Datei speichern
        output_path = os.path.join(output_folder, f"{safe_title}_anonymized.txt")
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(anonymized_text)

        print(f"💾 Anonymisierter Text gespeichert unter: {output_path}")


for filename, text_content in file_texts.items():
    print(f"\n📄 Originaltext ({filename}):\n{text_content}")

    # Optimierte Anonymisierung in einem Durchlauf
    pre_masked_text = mask_regex_patterns(text_content)
    text_with_names_masked = mask_names_with_spacy(pre_masked_text)
    anonymized = postprocess_domain_specific_pii(mask_pii(text_with_names_masked, aggregate_redaction=False))

    # Kompakter Output
    output_anonymization_results(f"{filename} (Optimized)", text_content, anonymized)





📄 Originaltext (2022 (1).txt):
22.01.2022
Mai 2022

📄 2022 (1).txt (Optimized) - Originaltext:
22.01.2022
Mai 2022

🔐 2022 (1).txt (Optimized) - Anonymisierter Text:
22.01.2022
Mai 2022



# **Level 2**